In [13]:
import pandas as pd
import networkx as nx

# Load books and user ratings datasets
books = pd.read_csv('datasets/book/book1-100k.csv').head(10000)

# Choose columns used for the algorithm
books = books[['Id', 'Name', 'Authors', 'Rating', 'pagesNumber']]

# Normalize book names
books['Name'] = books['Name'].str.strip().str.lower()

# Function to get liked books from user input
def get_liked_books():
    liked_books = []
    print("Enter the names of the books you like (type 'done' when finished):")
    while True:
        book_name = input("Book name: ").strip().lower()
        if book_name == 'done':
            break
        liked_books.append(book_name)
    return liked_books

liked_books = get_liked_books()

# Filter the books that match the liked books
liked_books_df = books[books['Name'].isin(liked_books)]

# Create graph
G = nx.Graph()

# Add book nodes
for _, row in books.iterrows():
    G.add_node(row['Id'], type='book', name=row['Name'], 
               authors=row['Authors'], rating=row['Rating'], pages=row['pagesNumber'])


# Add user node and edges based on the liked books
user_node = "user_1"  # You can change the user identifier as needed
G.add_node(user_node, type='user')

# Add an edge between the user and the liked books
for _, row in liked_books_df.iterrows():
    book_node = row['Id']
    G.add_edge(user_node, book_node)

# Calculate the mean pages of books liked by the user
total_pages = 0
books_count = 0
authors_set = set()

# Sum the pages of the books the user likes
for _, row in liked_books_df.iterrows():
    total_pages += row['pagesNumber']
    books_count += 1
    authors_set.update(row['Authors'].split(","))

# Save mean pages and authors for the user
if books_count > 0:
    mean_pages = total_pages / books_count
else:
    mean_pages = 0

# Display the results
print(f"\nUser's mean pages of liked books: {mean_pages}")
print(f"User's authors of liked books: {', '.join(authors_set)}")


In [26]:
def recommend_books_for_user():
    
    # Get the mean pages for the user
    user_mean = mean_pages
    user_author_set = authors.set
    
    # Calculate scores for each book
    book_scores = {}
    for book_node in G.nodes:
        if G.nodes[book_node].get('type') == 'book':
            # Get the global rating and pages of the book
            book_rating = G.nodes[book_node]['rating']
            book_pages = G.nodes[book_node]['pages'] 
            
            if abs(book_pages - user_mean) < 150:
                page_similarity = 1
            elif abs(book_pages - user_mean) < 300 and abs(book_pages - user_mean) > 150:
                page_similarity = 0.5
            else:
                page_similarity = 0
            
            score = book_rating + page_similarity
            
            # Add bonus for authors match
            author_bonus = 0
            book_authors = set(G.nodes[book_node]['authors'].split(","))
            if book_authors & user_author_set:  
                author_bonus = 1
            
            score += author_bonus  
            
            book_scores[book_node] = score
    
    # Sort books by score
    recommended_books = sorted(book_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get the top N recommended books
    top_recommended_books = [book[0] for book in recommended_books[:5]]
    
    return top_recommended_books


In [27]:
def display_recommendations():
    print(f"\nRecommended Books for User\n")
    
    for idx, book in enumerate(recommended_books, 1):
        book_name = G.nodes[book]['name']
        book_authors = G.nodes[book]['authors']
        print(f"{idx}. {book_name}")
        print(f"   Authors: {book_authors}")
        print("=" * 50)

display_recommendations()



Recommended Books for User 1:

1. bullets, badges, and bullshit
   Authors: Phil T. O'Brien
2. the real estate investor's tax guide
   Authors: Vernon Hoven
3. singers, heroes, and gods in the odyssey
   Authors: Charles Segal
4. greek literature in the classical period: the prose of historiography and oratory: greek literature
   Authors: Gregory Nagy
5. greek literature in the roman period and in late antiquity: greek literature
   Authors: Gregory Nagy
